## WeRateDogs - Query Twitter Data

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime

import tweepy
import requests
import json

In [65]:
df = pd.read_csv('twitter-archive-enhanced.csv', index_col=['tweet_id'], parse_dates=['timestamp'])

In [66]:
df.head(1)

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
tweet_id,,,,,,,,,,,,,,,,
892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 892420643555336193 to 666020888022790149
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   in_reply_to_status_id       78 non-null     float64            
 1   in_reply_to_user_id         78 non-null     float64            
 2   timestamp                   2356 non-null   datetime64[ns, UTC]
 3   source                      2356 non-null   object             
 4   text                        2356 non-null   object             
 5   retweeted_status_id         181 non-null    float64            
 6   retweeted_status_user_id    181 non-null    float64            
 7   retweeted_status_timestamp  181 non-null    object             
 8   expanded_urls               2297 non-null   object             
 9   rating_numerator            2356 non-null   int64              
 10  rating_denominator          2

In [68]:
?tw.API

In [69]:
access_token = 'xxxxx'
access_secret = 'xxxxx'
account_key = 'xxxxx'
account_secret = 'xxxxx'

auth = tweepy.OAuthHandler(account_key, account_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication successful")
except:
    print("Authentication error")

Authentication successful


In [70]:
id_list = list(df.index)

In [71]:
?api.get_status

In [59]:
start_date = datetime.now()

tweet_data = []
tweep_errors = []

for x in id_list:    
    try:
        tweet_data.append({x:api.get_status(x, tweet_mode='extended')._json})
    except tweepy.TweepError as error:
        print('Unable to query:', str(x))
        tweep_errors.append({x:error})

print(str(len(tweep_errors)/len(tweet_data)),'% of Tweets Missing')

with open('tweet_json.txt', 'w') as x:
    json.dump(tweet_data, x)
    
print(str(datetime.now() - start_date))

Unable to query: 888202515573088257
0.047619047619047616 % of Tweets Missing


In [60]:
x = api.get_status(id_list[0], tweet_mode='extended')
type(x)

tweepy.models.Status

In [40]:
x

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 

https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/

https://www.programcreek.com/python/example/13279/tweepy.TweepError

